## Neuron

In [10]:
import numpy as np

class Neuron(object):
    def __init__(self,num_inputs,activation_fn):
        self.W=np.random.rand(num_inputs) # weight
        self.b=np.random.rand(1) # bias
        self.activation_fn=activation_fn
        
    def forward(self,x):
        z=np.dot(x,self.W) + self.b
        return self.activation_fn(z)

np.random.seed(42)

x=np.random.rand(3).reshape(1,3)

step_fn=lambda y: 0 if y <=0 else 1 # step function

perceptron=Neuron(num_inputs=x.size,activation_fn=step_fn)
print(perceptron.W)
out=perceptron.forward(x)
print(out)

[0.59865848 0.15601864 0.15599452]
1


In [11]:
import numpy as np

class FullyConnectLayer(object):
    def __init__(self,num_inputs,layer_size,activation_fn):
        self.W=np.random.standard_normal((num_inputs,layer_size)) # weight
        self.b=np.random.rand(layer_size) # bias
        self.size=layer_size
        self.activation_fn=activation_fn
        
    def forward(self,x):
        z=np.dot(x,self.W) + self.b
        return self.activation_fn(z)
np.random.seed(42)
x1=np.random.uniform(-1,1,2).reshape(1,2)
print(x1)
x2=np.random.uniform(-1,1,2).reshape(1,2)
print(x2)

relu_fn=lambda y: np.maximum(y,0) #relu functioin
layer=FullyConnectLayer(2,3,relu_fn)
out1=layer.forward(x1)
print(out1)
out2=layer.forward(x2)
print(out2)

x12=np.concatenate((x1,x2))
out12=layer.forward(x12)
print(out12)

[[-0.25091976  0.90142861]]
[[0.46398788 0.19731697]]
[[0.9323663  0.         0.39706569]]
[[0.22460853 0.         1.14403416]]
[[0.9323663  0.         0.39706569]
 [0.22460853 0.         1.14403416]]


## Using MNIST

In [12]:
import tensorflow as tf
np.random.seed(42)
data_train, data_test = tf.keras.datasets.mnist.load_data()

In [13]:
(x_train,y_train)=data_train
(x_test,y_test)=data_test
num_classes=10

print('train',x_train.shape)
print('test',x_test.shape)
x_train,x_test=x_train.reshape(-1,28*28),x_test.reshape(-1,28*28)
print('after reshape')
print('train',x_train.shape)
print('test',x_test.shape)
y_train=np.eye(num_classes)[y_train]
print(y_train[0])

train (60000, 28, 28)
test (10000, 28, 28)
after reshape
train (60000, 784)
test (10000, 784)
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [20]:
def sigmoid(value):
    return 1.0/ (1.0 + np.exp(-value))

class SimpleNN(object):
    
    def __init__(self,num_inputs,num_outputs,hidden_layers_size=(64,32)):
        super().__init__()
        sizes=[num_inputs,*hidden_layers_size,num_outputs] # NeuralNet
        self.layers=[FullyConnectLayer(sizes[i],sizes[i+1],sigmoid) for i in range(len(sizes)-1)]
        
    def forward(self,x):
        for layer in self.layers:
            x=layer.forward(x)
        return x
    
    def predict(self,x):
        estimations=self.forward(x)
        best_class=np.argmax(estimations)
        return best_class
    
    def evaluate_accuracy(self,x_val,y_val): # valid test
        num_corrects=0
        for i in range(len(x_val)):
            if self.predict(x_val[i])==y_val[i]:
                num_corrects+=1
            return num_corrects/len(x_val)

In [21]:
mnist_classifier=SimpleNN(x_train.shape[1],num_classes,[64,32])

In [22]:
accuracy=mnist_classifier.evaluate_accuracy(x_test,y_test)
print('accuracy',accuracy)

accuracy 0.0


C:\Users\USER\AppData\Local\Temp/ipykernel_6160/578090442.py:2: RuntimeWarning: overflow encountered in exp
  return 1.0/ (1.0 + np.exp(-value))


In [23]:
class FullyConnectedLayer(object):
    def __init__(self,num_inputs,layer_size,activation_fn,d_activation_fn):
        self.W=np.random.standard_normal((num_inputs,layer_size)) # weight
        self.b=np.random.rand(layer_size) # bias
        self.size=layer_size
        self.activation_fn=activation_fn
        self.d_activation_fn=d_activation_fn
        self.x,self.y,self.dL_dW,self.dL_db=0,0,0,0
        
    
    def forward(self,x):
        z=np.dot(x,self.W) + self.b
        self.y=self.activation_fn(z)
        self.x=x
        return self.y
    
    def backward(self,dL_dy):
        dy_dz=self.d_activation_fn(self.y)
        dL_dz=(dL_dy*dy_dz)
        dz_dw=self.x.T
        dz_dx=self.W.T
        dz_db=np.ones(dL_dy.shape[0])
        self.dL_dW=np.dot(dz_dw,dL_dz)
        self.dL_db=np.dot(dz_db,dL_dz)
        dL_dx=np.dot(dL_dz,dz_dx)
        return dL_dx
    
    def optimize(self,epsilon):
        self.W-=epsilon*self.dL_dW
        self.b-=epsilon*self.dL_db
        

In [31]:
def derivated_sigmoid(y):
    return y*(1-y)

def loss_L2(pred,target):
    return np.sum(np.square(pred-target)) / pred.shape[0]

def derivated_loss_L2(pred,target):
    return 2*(pred-target)

class SimpleNetwork(object):
    def __init__(self,num_inputs,num_outputs,hidden_layers_size=(64,32),loss_fn=loss_L2,d_loss_fn=derivated_loss_L2):
        super().__init__()
        sizes=[num_inputs,*hidden_layers_size,num_outputs] # NeuralNet
        self.layers=[FullyConnectedLayer(sizes[i],sizes[i+1],sigmoid,derivated_sigmoid) for i in range(len(sizes)-1)]
        self.loss_fn,self.d_loss_fn=loss_fn,d_loss_fn
        
    def forward(self,x):
        for layer in self.layers:
            x=layer.forward(x)
        return x
    
    def predict(self,x):
        estimations=self.forward(x)
        best_class=np.argmax(estimations)
        return best_class
    
    def evaluate_accuracy(self,x_val,y_val): # valid test
        num_corrects=0
        for i in range(len(x_val)):
            if self.predict(x_val[i])==y_val[i]:
                num_corrects+=1
#             print(num_corrects)
            return num_corrects/len(x_val)
        
    def backward(self,dL_dy):
        for layer in reversed(self.layers):
            dL_dy=layer.backward(dL_dy)
        return dL_dy
    
    def optimize(self,epsilon):
        for layer in self.layers:
            layer.optimize(epsilon)
        
    def train(self,x_train,y_train,x_val,y_val,batch_size=32,num_epochs=5,learning_rates=5e-3):
        num_batches_per_epoch=len(x_train)//batch_size
        loss,accuracy=[],[]
        for i in range(num_epochs):
            epoch_loss=0
            for b in range(num_batches_per_epoch):
                b_idx=b*batch_size
                b_idx_e=b_idx+batch_size
                x,y_true=x_train[b_idx:b_idx_e],y_train[b_idx:b_idx_e]
                y=self.forward(x)
                epoch_loss+=self.loss_fn(y,y_true)
                dL_dy=self.d_loss_fn(y,y_true)
                self.backward(dL_dy)
                self.optimize(learning_rates)
            loss.append(epoch_loss/num_batches_per_epoch)
            accuracy.append(self.evaluate_accuracy(x_val,y_val))
            print("Epoch",i,"loss",loss[i],'accuracy',accuracy[i]*100)
        
        return loss,accuracy
            

In [32]:
mnist_classifier=SimpleNetwork(x_train.shape[1],num_classes,[64,32])
losses,accuarcies=mnist_classifier.train(x_train,y_train,x_test,y_test,batch_size=30,num_epochs=500)

C:\Users\USER\AppData\Local\Temp/ipykernel_6160/578090442.py:2: RuntimeWarning: overflow encountered in exp
  return 1.0/ (1.0 + np.exp(-value))


Epoch 0 loss 1.6016706026570116 accuracy 0.0
Epoch 1 loss 0.8506987255459663 accuracy 0.01
Epoch 2 loss 0.49663140106598014 accuracy 0.01


KeyboardInterrupt: 